In [4]:
import face_recognition
import cv2
import numpy as np

In [5]:
# 获取对网络摄像头的引用
video_capture = cv2.VideoCapture(0)

# 加载样本图片并学习如何识别它
obama_image = face_recognition.load_image_file("../image/obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# 加载第二章样本图片并学习如何识别它
liudehua_image = face_recognition.load_image_file("../image/yangmi.jpg")
liudehua_face_encoding = face_recognition.face_encodings(liudehua_image)[0]

# 创建已知面部编码及其名称的数组
known_face_encodings = [
    obama_face_encoding,
    liudehua_face_encoding
]
known_face_names = [
    "奥巴马",
    "杨幂"
]

# 初始化一些变量
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    #     抓住一帧视频
    ret, frame = video_capture.read()
    #     将视频帧大小调整为四分之一大小，以便更快进行人脸识别处理
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    #     将图像从BGR颜色转换为RGB颜色
    rgb_small_frame = small_frame[:, :, ::-1]
    #  仅处理每隔一帧视频以节省时间
    if process_this_frame:
        # 查找当前视频帧中的所有面部和面部代码
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        face_names = []
        for face_encoding in face_encodings:
            #  查看面部是否与已知面部匹配
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            # 或者使用距离新面最小距离的已知面部
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            
            face_names.append(name)
    process_this_frame = not process_this_frame
    
#     结果展示
    for(top, right, bottom, left), name in zip(face_locations, face_names):
        #  缩放脸部位置，因为我们检测道德帧被缩放到四分之一大小
        top *= 4
        right *= 4 
        bottom *= 4 
        left *= 4
        
        #  在脸上画一个盒子
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        
        # 在脸部下面画一个名称标签
        cv2.rectangle(frame, (left, bottom-35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left+6, bottom-6), font, 1.0, (255, 255, 255), 1)
        
    #  显示生成的图像
    cv2.imshow("video", frame)
    
    
    #  在键盘上敲出q退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
#     释放网络摄像头的手柄
video_capture.release()
cv2.destroyAllWindows()
            
            
      

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
